In [210]:
import numpy as np
import pandas as pd
import sklearn as sk
import librosa
import glob
import os
from sklearn import svm

from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn import neighbors
from sklearn import decomposition
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import precision_recall_fscore_support
#import Preprocessing

C:\Users\Admin\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Admin\Anaconda2\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [215]:
def preprocess_traindata():
    music_data = []
    #reading all files recursively
    files = glob.glob('Datasets\TrainingData\*\*.wav')
    np.random.shuffle(files)
    
    for filename in files:
        #Preprocess the music file to get features from it
        music, sr = librosa.load(filename)
        mfccs = librosa.feature.mfcc(y=music, sr=sr)
        mean_mfccs = np.mean(mfccs, axis = 1)
        feature = mean_mfccs.reshape(20)
        if '[cel]' in filename[25:38]:
            instrument_code = 1
        elif '[flu]' in filename[25:38]:
            instrument_code = 2
        elif '[gac]' in filename[25:38]:
            instrument_code = 3
        elif '[gel]' in filename[25:38]:
            instrument_code = 4
        elif '[org]' in filename[25:38]:
            instrument_code = 5
        elif '[pia]' in filename[25:38]:
            instrument_code = 6
        elif '[sax]' in filename[25:38]:
            instrument_code = 7
        elif '[tru]' in filename[25:38]:
            instrument_code = 8
        elif '[vio]' in filename[25:38]:
            instrument_code = 9
        elif '[cla]' in filename[25:38]:
            instrument_code = 10
        elif '[voi]' in filename[25:38]:
            instrument_code = 11
        else:
            instrument_code = 0
            print('Unknown instrument found in the file', filename)
        #Append the filename, feature and instrument list of each file into a list
        #filelist.append(filename)
        #featurelist.append(feature)
        #inst_code.append(instrument)
        #Copy the list into a dataframe
        #df=pd.DataFrame()    
        #df['filename']= filelist
        #df['feature']= featurelist
        #df['instrumentcode']= inst_code
        file_data = [filename, feature, instrument_code]
        music_data.append(file_data)

    return music_data
    

In [162]:
def preprocess_testdata():
    music_data = []

    #reading all files recursively
    files = glob.glob('Datasets\TestingData-Part1\Part1\*.wav')
    np.random.shuffle(files)

    for filename in files:
        #Preprocess the music file to get features from it
        music, sr = librosa.load(filename)
        mfccs = librosa.feature.mfcc(y=music, sr=sr)
        mean_mfccs = np.mean(mfccs, axis = 1)
        feature = mean_mfccs.reshape(20)
        # Open the corresponding text file containing list of instruments.
        instrument_file = os.path.splitext(filename)[0]+'.txt'
        #CHECK IF THE FILE IS TAKEN FROM THE RIGHT PATH
        f = open(instrument_file, 'r')
        instrument = []
        for word in f:
            if 'cel' in word:
                instrument_code = 1
            elif 'cla' in word:
                instrument_code = 2
            elif 'flu' in word:
                instrument_code = 3
            elif 'gac' in word:
                instrument_code = 4
            elif 'gel' in word:
                instrument_code = 5
            elif 'org' in word:
                instrument_code = 6
            elif 'pia' in word:
                instrument_code = 7
            elif 'sax' in word:
                instrument_code = 8
            elif 'tru' in word:
                instrument_code = 9
            elif 'vio' in word:
                instrument_code = 10
            elif 'voi' in word:
                instrument_code = 11
            else:
                instrument_code = 0
            instrument.append(instrument_code)
        #Append the filename, feature and instrument list of each file into a list
        #filelist.append(filename)
        #featurelist.append(feature)
        #inst_code.append(instrument)
        #Copy the list into a dataframe
        #df=pd.DataFrame()    
        #df['filename']= filelist
        #df['feature']= featurelist
        #df['instrumentcode']= inst_code
        file_data = [filename, feature, instrument]
        music_data.append(file_data)

        #return df
    return music_data
    

In [163]:
def unpack_data(data):
    filename = np.array(map(lambda n: n[0], data))
    feature = np.array(map(lambda n: n[1], data))
    instrument = np.array(map(lambda n: n[2], data))
    return filename, feature, instrument

In [221]:
def train(data):
    #Copy the parameters into a dataframe
    #df = pd.DataFrame()
    #df['feature'] = features
    #df['instrumentcode'] = instrument_code
    #Copying the paramters into an array
    #features = np.array(features)
    #instrument_code = np.array(instrument_code)
    filename, features, instrument_code = unpack_data(data)
    
    X = preprocessing.Imputer().fit_transform(features)
    le = preprocessing.LabelEncoder()
    y = le.fit_transform(instrument_code)
    estimators = [("scale", preprocessing.StandardScaler()),
                  ('svm', svm.SVC(decision_function_shape='ovo'))]
    
    clf = Pipeline(estimators)
    params = dict(svm__kernel=['rbf'], svm__C=[0.1],
                  svm__degree=[1, 3], svm__gamma=[0.01])
    gs = GridSearchCV(clf, param_grid=params, cv=10, verbose=2)
    gs.fit(X, y)
    prediction = gs.predict(X)
    
    Evaluate_accuracy(prediction, y)
    #PCA
    #pca = decomposition.PCA(n_components=10)
    #features = pca.fit_transform(features)
    
    #SVM
    #svc = svm.LinearSVC()
    #svc.fit(features, instrument_code)
    
    #KNN
    #knn = neighbors.KNeighborsClassifier(n_neighbors=12)
    #knn.fit(features, instrument_code)

    #return svc
    return gs

In [165]:
def predict(model, data):
    filename, features, instrument_code = unpack_data(data)
    
    #PCA
    #features = pca.transform(features)
    
    #SVM
    prediction = model.predict(features)
    
    #mlb = MultiLabelBinarizer()
    #instrument_code = mlb.fit_transform(instrument_code)/mlb.transform(instrument_code)
    
    #KNN
    #prediction = model.predict(features)

    #Evaluate the performance
    Evaluate_accuracy(prediction, instrument_code)
    return 

In [204]:
def Evaluate_accuracy(pred, true_value):
    #Evaluate the accuracy of the model

    print("Accuracy score is", accuracy_score(true_value.astype(int), pred.astype(int))*100)
    print("Mean squared error", mean_squared_error(true_value, pred))
    rmse = np.sqrt(mean_squared_error(true_value, pred))
    print("Root Mean Squared Error: {}".format(rmse))
    print("Mean absolute error:", mean_absolute_error(true_value,pred))
    print("Classification Report: ",classification_report(true_value, pred))
    print('confusion matrix:', confusion_matrix(true_value, pred))
    print "Micro stats:"
    print precision_recall_fscore_support(true_value, pred, average='micro')
    print "Macro stats:"
    print precision_recall_fscore_support(true_value, pred, average='macro')
    return

In [212]:
def saveModel(model):
    joblib.dump(model, 'gsclsclassifier.model')
    #joblib.dump(pca, 'pcaclassifier.model')
    #joblib.dump(model, 'svmclassifier.model')
    #joblib.dump(knn, 'knnclassifier.model')
    return

In [213]:
def loadModel():
    #model = joblib.load('newclassifier.model')
    #pca = joblib.load('pcaclassifier.model')
    svc = joblib.load('svmclassifier.model')
    #knn = joblib.load('knnclassifier.model')
    return svc

In [214]:
def main():
    train_data = preprocess_traindata()

    model= train(train_data)
    saveModel(model)

    #model = loadModel()
    #test_data = preprocess_testdata()
    #test_data = preprocess_traindata()
    #predict(model, test_data)


In [222]:
if __name__ == '__main__':
    main()

Fitting 10 folds for each of 2 candidates, totalling 20 fits
[CV] svm__degree=1, svm__C=0.1, svm__kernel=rbf, svm__gamma=0.01 .....
[CV]  svm__degree=1, svm__C=0.1, svm__kernel=rbf, svm__gamma=0.01 -   9.1s
[CV] svm__degree=1, svm__C=0.1, svm__kernel=rbf, svm__gamma=0.01 .....


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.1s remaining:    0.0s


[CV]  svm__degree=1, svm__C=0.1, svm__kernel=rbf, svm__gamma=0.01 -   3.7s
[CV] svm__degree=1, svm__C=0.1, svm__kernel=rbf, svm__gamma=0.01 .....
[CV]  svm__degree=1, svm__C=0.1, svm__kernel=rbf, svm__gamma=0.01 -   3.8s
[CV] svm__degree=1, svm__C=0.1, svm__kernel=rbf, svm__gamma=0.01 .....
[CV]  svm__degree=1, svm__C=0.1, svm__kernel=rbf, svm__gamma=0.01 -   3.8s
[CV] svm__degree=1, svm__C=0.1, svm__kernel=rbf, svm__gamma=0.01 .....
[CV]  svm__degree=1, svm__C=0.1, svm__kernel=rbf, svm__gamma=0.01 -   3.7s
[CV] svm__degree=1, svm__C=0.1, svm__kernel=rbf, svm__gamma=0.01 .....
[CV]  svm__degree=1, svm__C=0.1, svm__kernel=rbf, svm__gamma=0.01 -   3.7s
[CV] svm__degree=1, svm__C=0.1, svm__kernel=rbf, svm__gamma=0.01 .....
[CV]  svm__degree=1, svm__C=0.1, svm__kernel=rbf, svm__gamma=0.01 -   3.7s
[CV] svm__degree=1, svm__C=0.1, svm__kernel=rbf, svm__gamma=0.01 .....
[CV]  svm__degree=1, svm__C=0.1, svm__kernel=rbf, svm__gamma=0.01 -   3.8s
[CV] svm__degree=1, svm__C=0.1, svm__kernel=rbf, 

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  1.4min finished


('Accuracy score is', 38.031319910514547)
('Mean squared error', 12.879940343027592)
Root Mean Squared Error: 3.58886337759
('Mean absolute error:', 2.410439970171514)
('Classification Report: ', '             precision    recall  f1-score   support\n\n          0       0.78      0.08      0.14       388\n          1       0.56      0.07      0.12       451\n          2       0.40      0.52      0.45       637\n          3       0.35      0.44      0.39       760\n          4       0.37      0.56      0.45       682\n          5       0.41      0.56      0.47       721\n          6       0.30      0.23      0.26       626\n          7       0.51      0.27      0.35       577\n          8       0.45      0.28      0.35       580\n          9       0.51      0.19      0.28       505\n         10       0.33      0.61      0.42       778\n\navg / total       0.43      0.38      0.35      6705\n')
('confusion matrix:', array([[ 31,   2,  89,  52,  19,  58,  36,   1,  63,   4,  33],
       [

On testing with training data itself:
When using KNN:
('Accuracy score is', 62.4750499001996)
('Mean squared error', 7.3732534930139719)
Root Mean Squared Error: 2.71537354576
('Mean absolute error:', 1.4131736526946108)
When using SVM:
('Number of filessssssssssssss trained', 1001)
('Accuracy score is', 14.685314685314685)
('Mean squared error', 31.002997002997002)
Root Mean Squared Error: 5.56803349514
('Mean absolute error:', 4.615384615384615)

when using this code:
    X = preprocessing.Imputer().fit_transform(features)
    le = preprocessing.LabelEncoder()
    y = le.fit_transform(instrument_code)
    estimators = [("scale", preprocessing.StandardScaler()),
                  ('svm', svm.SVC(decision_function_shape='ovo'))]
    
    clf = Pipeline(estimators)
    params = dict(svm__kernel=['rbf'], svm__C=[0.1],
                  svm__degree=[1, 3], svm__gamma=[0.01])
    gs = GridSearchCV(clf, param_grid=params, cv=10, verbose=2)
    gs.fit(X, y)
    prediction = gs.predict(X)
    
('Accuracy score is', 38.031319910514547)
('Mean squared error', 12.879940343027592)
Root Mean Squared Error: 3.58886337759
('Mean absolute error:', 2.410439970171514)
('Classification Report: ', '             precision    recall  f1-score   support\n\n          0       0.78      0.08      0.14       388\n          1       0.56      0.07      0.12       451\n          2       0.40      0.52      0.45       637\n          3       0.35      0.44      0.39       760\n          4       0.37      0.56      0.45       682\n          5       0.41      0.56      0.47       721\n          6       0.30      0.23      0.26       626\n          7       0.51      0.27      0.35       577\n          8       0.45      0.28      0.35       580\n          9       0.51      0.19      0.28       505\n         10       0.33      0.61      0.42       778\n\navg / total       0.43      0.38      0.35      6705\n')
('confusion matrix:', array([[ 31,   2,  89,  52,  19,  58,  36,   1,  63,   4,  33],
       [  2,  31,  53,  21,  98,  86,  21,  11,  27,  27,  74],
       [  0,   0, 333,  44,  66,  60,  19,   4,   4,   0, 107],
       [  1,   1,  37, 337,  81,  56,  25,   6,  15,   0, 201],
       [  1,   0,  30,  96, 380,  13,   5,  15,   4,   2, 136],
       [  1,   1,  42,  43, 107, 402,  43,  11,   5,   4,  62],
       [  0,   3,  51,  71,  50,  87, 146,  63,  13,  15, 127],
       [  2,   5,  16,  55,  62,  70,  52, 156,  26,  27, 106],
       [  1,   5,  61,  80,  37,  53,  64,  10, 165,  11,  93],
       [  1,   7,  72,  47,  22,  96,  67,  22,  37,  96,  38],
       [  0,   0,  50, 114, 102,   8,  13,   6,   9,   3, 473]]))
Micro stats:
(0.38031319910514544, 0.38031319910514544, 0.38031319910514544, None)
Macro stats:
(0.45071493277784475, 0.34688204167794512, 0.33565231678868362, None)